## Define Variables / Import MetaData

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from cartopy import crs as ccrs 
import cartopy.feature as cfeature
import pandas as pd
import hvplot.pandas
import xarray as xr
import hvplot.xarray
import geoviews.feature as gf

In [ ]:
# Define misc variables 
# I can't get the config.py to work in jupyternotebook because it does not know where $NOBACKUP is
amer_filepath = '../../ameriflux-data/'
mic_filepath = '../intermediates/'

In [ ]:
# Import site metadata csv
meta_file = amer_filepath + 'AmeriFlux-site-search-results-202410071335.tsv'
ameriflux_meta = pd.read_csv(meta_file, sep='\t')
fluxnet_meta = ameriflux_meta.loc[ameriflux_meta['AmeriFlux FLUXNET Data'] == 'Yes'] 

In [ ]:
# set map proj
proj=ccrs.PlateCarree()

## Import metadata and RSME values for NEE

In [ ]:
# Create df with lat/lons
site_subset = ['Site ID', 
               'Longitude (degrees)',
                'Latitude (degrees)',
               ]
df_meta = fluxnet_meta[site_subset]
df_meta.set_index('Site ID');

In [ ]:
value='NEE_RSME'

In [ ]:
# Import and merge results
results = pd.read_csv('../analysis/results.csv',index_col='SiteID')[value]

In [ ]:
df = df_meta.join(results, on='Site ID')
df = df.rename(columns={'Latitude (degrees)': 'lat', 'Longitude (degrees)': 'lon'})
df

### Subset North America

In [ ]:
NA_mask = (df['lat']>=0)
df = df[NA_mask]

In [ ]:
df

### Xarray Matplotlib View

In [ ]:
ds = xr.Dataset(
    coords={
        'site_id': df['Site ID'].values,
        'lat': ('site_id', df['lat'].values),
        'lon': ('site_id', df['lon'].values),
    }, 
    data_vars={
        value: ("site_id", df[value].values),
    }
)
ds

In [ ]:
import cartopy
print(cartopy.config['data_dir'])


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8), subplot_kw={'projection': proj}, constrained_layout=True);
ax.add_feature(cfeature.LAND,
               edgecolor='lightgrey',facecolor='none',
               zorder=0
              )
ax.add_feature(cfeature.STATES,edgecolor='lightgrey', zorder=0)

plot = ds.plot.scatter(x="lon", y="lat",ax=ax,
                       s=40,
                       markersize=value, edgecolor='none',add_legend=False,
                        norm=colors.LogNorm(), 
                       hue=value,
                       cmap='autumn_r',
                       # cmap='cool',
                       add_colorbar=False
                      )

cbar = fig.colorbar(plot, ax=ax, shrink=0.9, label=value[4:], orientation='horizontal')
ax.set_title('NEE RSME (North America Only)')
plt.show()

## Select RSME>10-7

In [ ]:
ds_sel = ds.where(ds[value]>10**-7,drop=True)
ds_sel

In [ ]:
from adjustText import adjust_text

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,8), subplot_kw={'projection': proj}, constrained_layout=True);
ax.add_feature(cfeature.LAND,
               edgecolor='lightgrey',facecolor='none',
               zorder=0
              )
ax.add_feature(cfeature.STATES,edgecolor='lightgrey', zorder=0)

plot = ds_sel.plot.scatter(x="lon", y="lat",
                           ax=ax,
                           s=40,
                           edgecolor='black',add_legend=False,
                        # norm=colors.LogNorm(),  
                       hue=value,
                       cmap='autumn_r',
                       add_colorbar=False
                      )

texts = []
for i in range(len(ds_sel.site_id)):
    txt = ax.annotate(ds_sel.site_id.values[i], 
                (ds_sel.lon.values[i], ds_sel.lat.values[i]),
                transform=proj,
                fontsize=12)
    texts.append(txt)
adjust_text(texts, ax=ax)

cbar = fig.colorbar(plot, ax=ax, shrink=0.9, label=value[4:], orientation='horizontal')
# ax.set_title(value)
fig.suptitle('NEE RSME values Higher Than $10^{-7}$', y=0.85)
plt.show()

In [ ]:
# ds_sel.site_id.values.tolist()

In [ ]:
# df_sel = df[df[value]>=10**-7]

In [ ]:
# df_sel

In [ ]:
# def output_range(pd_series):
#     return pd_series.min()-5, pd_series.max()+5

In [ ]:
# min_lon, max_lon = output_range(df["lon"])
# min_lat, max_lat = output_range(df["lat"])

In [ ]:
# df_sel.hvplot.points(x="lon", 
#                 y="lat",
#                 geo=True, 
#                 # crs=ccrs.PlateCarree(),

#                 c=value,
#                 logz=True,
#                 cmap='autumn_r',
#                 clabel=f'{value}',

#                  size=45,
#                  # Size values don't scale logarithmically
#                 # s=values[0],
#                 # scale=4500,
#                  # color='red',
                
#                 tiles=True,
#                 tiles_opts={'alpha': 0.4},
#                 # tiles=xyz.Esri.WorldGrayCanvas,


#                 hover_cols=['Site ID'],

#                 # # width=700, height=500,
#                 # xlim=(min_lon, max_lon),   # longitude range
#                 # ylim=(min_lat, max_lat),     # latitude range
#                 # frame_width=800,
#                 frame_height=700
#                                    )